In [1]:
# Importing the relevant packages
import s3fs

import random as rn
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical
from keras.models import Sequential 
from keras.layers import Dense
from keras.regularizers import l2

from sklearn.model_selection import train_test_split

seed = 6
np.random.seed(6)
rn.seed(6)
tf.random.set_seed(6)

Using TensorFlow backend.
